# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 95421628,
 'calculationPrice': 'close',
 'change': -2.32,
 'changePercent': -0.01564,
 'close': 150.53,
 'closeSource': 'lffiocai',
 'closeTime': 1709451547640,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 153.15,
 'delayedPriceTime': 1703254522591,
 'extendedChange': 0.17,
 'extendedChangePercent': 0.00116,
 'extendedPrice': 156.3,
 'extendedPriceTime': 1731499382841,
 'high': 157.2,
 'highSource': ' eltnueaiiced5d1e mry p',
 'highTime': 1668790229173,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 158.06,
 'iexCloseTime': 1679583640567,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 156.01,
 'iexOpenTime': 1738257549033,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1682463932081,
 'latestPrice': 157.26,
 'latestSource': 'Close',
 'latestTime': 'September 23, 2022',
 'latestUpdate': 1674829452996,
 'latestVolume': 96929876,
 'low': 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ration = data['peRatio']
pe_ration


25.63

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW':
            continue
        final_dataframe = final_dataframe.append(
            pd.Series([symbol,
                       data[symbol]['quote']['latestPrice'],
                       data[symbol]['quote']['peRatio'],
                       'N/A'
                       
                       
                       
                       ],
                      index = my_columns),
            ignore_index = True
        )
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,123.52,28.86,N/A
1,AAL,12.62,-4.38,N/A
2,AAP,163.60,18.99,N/A
3,AAPL,150.49,25.13,N/A
4,ABBV,146.16,20.98,N/A
...,...,...,...,...
496,YUM,113.01,22.12,N/A
497,ZBH,110.70,99.97,N/A
498,ZBRA,278.81,29.32,N/A
499,ZION,57.18,6.18,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.94,2.89,N/A
1,AIG,50.59,3.35,N/A
2,NUE,106.49,3.41,N/A
3,PVH,50.80,3.85,N/A
4,APA,33.68,3.99,N/A
5,COF,98.27,4.2,N/A
6,SYF,29.73,4.33,N/A
7,F,12.90,4.33,N/A
8,HPQ,25.56,4.44,N/A
9,PHM,40.56,4.48,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [10]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [20]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

20000.0


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.94,2.89,476
1,AIG,50.59,3.35,395
2,NUE,106.49,3.41,187
3,PVH,50.80,3.85,393
4,APA,33.68,3.99,593
5,COF,98.27,4.2,203
6,SYF,29.73,4.33,672
7,F,12.90,4.33,1550
8,HPQ,25.56,4.44,782
9,PHM,40.56,4.48,493


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [22]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

#P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

#P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

#EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
#EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [26]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW':
            continue
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

In [27]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.31,N/A,28.21,N/A,7.5,N/A,5.53,N/A,20.528815,N/A,10.540368,N/A,N/A
1,AAL,12.41,N/A,-4.41,N/A,-0.9672,N/A,0.1975,N/A,103.129385,N/A,0.792252,N/A,N/A
2,AAP,160.80,N/A,18.73,N/A,3.61,N/A,0.8877,N/A,10.425250,N/A,2.115517,N/A,N/A
3,AAPL,151.56,N/A,25.92,N/A,43.1,N/A,6.3,N/A,19.893459,N/A,14.783189,N/A,N/A
4,ABBV,149.21,N/A,21.2,N/A,18.03,N/A,4.42,N/A,11.320625,N/A,7.779424,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,115.29,N/A,23.17,N/A,-3.77,N/A,4.81,N/A,18.628207,N/A,8.476691,N/A,N/A
497,ZBH,108.09,N/A,101.18,N/A,1.88,N/A,3.08,N/A,17.883338,N/A,5.469483,N/A,N/A
498,ZBRA,270.31,N/A,29.51,N/A,5.54,N/A,2.42,N/A,14.997764,N/A,6.114030,N/A,N/A
499,ZION,57.21,N/A,6.18,N/A,1.64,N/A,2.45,N/A,4.921915,N/A,2.359379,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [28]:
rv_dataframe[rv_dataframe.isnull().any(axis = 1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,183.200,N/A,61.55,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,284.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,274.950,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,95.070,N/A,48.92,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.03,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.900,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.440,N/A,14.7,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.010,N/A,33.48,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,29.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,91.460,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [33]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio','EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [53]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,49.66,1006,3.86,0.12,0.6325,0.14,0.3615,0.24,4.135880,0.46,0.882089,0.22,5.668663
1,AIG,50.91,982,3.28,0.1,0.8718,0.34,0.6790,0.64,2.019631,0.02,0.666149,0.1,5.788423
2,KSS,27.00,1851,5.19,0.36,0.6959,0.16,0.1629,0.04,3.973824,0.44,1.052170,0.36,5.948104
3,F,12.68,3943,4.44,0.2,1.1000,0.51,0.3297,0.18,3.170753,0.2,1.705457,0.48,7.48503
4,GM,36.59,1366,6.99,0.66,0.8289,0.3,0.4076,0.34,2.072473,0.04,1.641885,0.46,7.864271
5,SYF,30.31,1649,4.33,0.16,1.1400,0.57,0.8044,0.72,2.724864,0.08,0.894926,0.24,8.083832
6,UNM,39.11,1278,7.34,0.74,0.8283,0.28,0.6500,0.6,4.140590,0.5,0.651484,0.08,8.463074
7,COF,95.57,523,4.20,0.14,0.7049,0.18,1.0700,0.89,2.815680,0.1,1.089140,0.38,8.602794
8,HPQ,26.55,1883,4.56,0.24,-11.0000,0.06,0.4042,0.32,5.437739,0.76,2.369996,0.6,9.021956
9,C,45.62,1096,5.73,0.42,0.4823,0.12,1.0600,0.86,3.268463,0.24,1.005124,0.32,9.101796


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [54]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row,metrics[metric]])
        
    rv_dataframe.loc[row,'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,49.66,1006,3.86,0.12,0.6325,0.14,0.3615,0.24,4.135880,0.46,0.882089,0.22,0.236
1,AIG,50.91,982,3.28,0.1,0.8718,0.34,0.6790,0.64,2.019631,0.02,0.666149,0.1,0.24
2,KSS,27.00,1851,5.19,0.36,0.6959,0.16,0.1629,0.04,3.973824,0.44,1.052170,0.36,0.272
3,F,12.68,3943,4.44,0.2,1.1000,0.51,0.3297,0.18,3.170753,0.2,1.705457,0.48,0.314
4,GM,36.59,1366,6.99,0.66,0.8289,0.3,0.4076,0.34,2.072473,0.04,1.641885,0.46,0.36
5,SYF,30.31,1649,4.33,0.16,1.1400,0.57,0.8044,0.72,2.724864,0.08,0.894926,0.24,0.354
6,UNM,39.11,1278,7.34,0.74,0.8283,0.28,0.6500,0.6,4.140590,0.5,0.651484,0.08,0.44
7,COF,95.57,523,4.20,0.14,0.7049,0.18,1.0700,0.89,2.815680,0.1,1.089140,0.38,0.338
8,HPQ,26.55,1883,4.56,0.24,-11.0000,0.06,0.4042,0.32,5.437739,0.76,2.369996,0.6,0.396
9,C,45.62,1096,5.73,0.42,0.4823,0.12,1.0600,0.86,3.268463,0.24,1.005124,0.32,0.392


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [55]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
#since we want the lowest Pto something as this means the most worth

rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,49.66,1006,3.86,0.12,0.6325,0.14,0.3615,0.24,4.135880,0.46,0.882089,0.22,0.236
1,AIG,50.91,982,3.28,0.1,0.8718,0.34,0.6790,0.64,2.019631,0.02,0.666149,0.1,0.24
2,KSS,27.00,1851,5.19,0.36,0.6959,0.16,0.1629,0.04,3.973824,0.44,1.052170,0.36,0.272
3,F,12.68,3943,4.44,0.2,1.1000,0.51,0.3297,0.18,3.170753,0.2,1.705457,0.48,0.314
4,COF,95.57,523,4.20,0.14,0.7049,0.18,1.0700,0.89,2.815680,0.1,1.089140,0.38,0.338
5,CAH,67.70,738,-20.43,0.02,-25.9900,0.02,0.1005,0.02,6.745110,0.9,2.843173,0.74,0.34
6,SYF,30.31,1649,4.33,0.16,1.1400,0.57,0.8044,0.72,2.724864,0.08,0.894926,0.24,0.354
7,GM,36.59,1366,6.99,0.66,0.8289,0.3,0.4076,0.34,2.072473,0.04,1.641885,0.46,0.36
8,NRG,42.43,1178,2.91,0.06,1.9000,0.96,0.3369,0.2,2.835517,0.12,2.317112,0.56,0.38
9,GPS,8.72,5733,-8.39,0.04,1.3800,0.7,0.2013,0.06,9.114671,0.98,0.753252,0.16,0.388


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [44]:
portfolio_input()

In [56]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,49.66,1006,3.86,0.12,0.6325,0.14,0.3615,0.24,4.135880,0.46,0.882089,0.22,0.236
1,AIG,50.91,982,3.28,0.1,0.8718,0.34,0.6790,0.64,2.019631,0.02,0.666149,0.1,0.24
2,KSS,27.00,1851,5.19,0.36,0.6959,0.16,0.1629,0.04,3.973824,0.44,1.052170,0.36,0.272
3,F,12.68,3943,4.44,0.2,1.1000,0.51,0.3297,0.18,3.170753,0.2,1.705457,0.48,0.314
4,COF,95.57,523,4.20,0.14,0.7049,0.18,1.0700,0.89,2.815680,0.1,1.089140,0.38,0.338
5,CAH,67.70,738,-20.43,0.02,-25.9900,0.02,0.1005,0.02,6.745110,0.9,2.843173,0.74,0.34
6,SYF,30.31,1649,4.33,0.16,1.1400,0.57,0.8044,0.72,2.724864,0.08,0.894926,0.24,0.354
7,GM,36.59,1366,6.99,0.66,0.8289,0.3,0.4076,0.34,2.072473,0.04,1.641885,0.46,0.36
8,NRG,42.43,1178,2.91,0.06,1.9000,0.96,0.3369,0.2,2.835517,0.12,2.317112,0.56,0.38
9,GPS,8.72,5733,-8.39,0.04,1.3800,0.7,0.2013,0.06,9.114671,0.98,0.753252,0.16,0.388


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [57]:
writer = pd.ExcelWriter('Value_Strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)



In [58]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [59]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [60]:
writer.save()